In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
import random
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kshort_pn_4GeV_Jun2_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x


In [4]:
def track_smear(kinetic_energy, theta):
    # assume flat uncertainty on the energy (I think that is 6%)
    c_eres = random.gauss(1.,0.06); # flat 10% uncertainty for now
    # assume some uncertainty on the angle (I think that is 2%)
    c_hres = random.gauss(0.,0.02); # 1 degrees
    return (c_eres*kinetic_energy, abs(c_hres+theta))

In [5]:
hists = {}
hists["Target_pion_ke"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"Kinetic Energy [MeV]", 80, 0, 2000),
                                hist.Bin("keSm", r"Kinetic Energy [MeV]", 80, 0, 2000),
                            )       

In [6]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    target_kaon_dau1 = ldmx_events['TSP_Kaon_dau1']
    target_kaon_dau2 = ldmx_events['TSP_Kaon_dau2']
    
    kshort_pdgid = 310
    chargedpion_pdgid = 211
    pion_mass = 139
    
    mask_2dau = (target_kaon_dau1.gd==0)
    kshort_mask = (abs(target_kaon_dau1.mompdgID)==kshort_pdgid)
    pidau1_mask = (abs(target_kaon_dau1.pdgID)==chargedpion_pdgid)

 
    pi1 = get_vector(target_kaon_dau1[pidau1_mask & kshort_mask & mask_2dau])

    #print(flat(pi1.theta).size)
    #print((flat(pi1.t) - pion_mass).size)
    
    new_energy1 = track_smear(pi1.t - pion_mass, pi1.theta)[0] + pion_mass
    new_theta1 = track_smear(pi1.t - pion_mass, pi1.theta)[1]
    
   # print(new_energy1)
    #print(pi1.t)
    
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    pi1_track_smeared =  ak.zip({"x": np.cos(new_theta1)*pi1.p,
                    "y": np.sin(new_theta1)*pi1.p,
                    "z": pi1.pz,
                    "t": new_energy1,
                    },
                    with_name="LorentzVector")
    
    #print(flat(pi1.theta))
    #print(flat(new_theta1)) these two pump out reasonable things
    
    #print(flat(np.cos(pi1.theta)))
    #print(flat(np.cos(new_theta1))) #these two pump out reasonable things
    
   # print(flat(pi1.px))

   # print(flat(np.cos(new_theta1))*flat(pi1.p))
    
               #+flat(pi1.y)^2+flat(pi1.z)^2)
    
   # print(flat(np.cos(pi1.theta)*pi1.p))
    #print(flat(np.cos(pi1.theta)* momentum))
    
#show that px is p * theta 
#use vector that we know pi1, print pi1.px and pi1.theta, take the cos of that and multipy pi1.p
#see if track smeared can work without subtracting pion mass
    
    
    
 #   print(flat(pi1.mass), flat(new_energy1), flat(pi1.t))
    
   # hists["Target_pion_ke"].fill(
                   #              keOG=flat(new_energy1)-pion_mass,
                        #         keSm=c_eres*kinetic_energy,
                       #         )
   # print()

    
    
    return

In [7]:
nchunk = 0
for chunk in ldmx_dict:
    #if nchunk >= 1: break
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

process 1


ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
print(hists["Target_pion_ke"])
fig, ax = plt.subplots()
#fig, ax = plt.subplots(1,2,figsize=(24,6), constrained_layout=True)
hist.plot1d(hists["Target_pion_ke"],ax=ax,clear=False);
leg = ax.legend([r'Dau1'])

#hist.plot1d(hists["Target_pion_ke_new"].sum('ke1'),ax=ax[1],clear=False);
#hist.plot1d(hists["Target_pion_ke_new"].sum('ke2'),ax=ax[1],clear=False);
#leg = ax[1].legend([r'Dau1 with Mass Smear', r'Dau2 with Mass Smear'])